# Analysis for Holmusk data set

## Data processing

In [87]:
# Import some libraries
import pandas as pd
import numpy as np
import os

In [88]:
cwd = os.getcwd()
print(cwd)

C:\Users\USER\Downloads\Holmusk_analysis


In [89]:
# Import our datasets
bill_id = pd.read_csv(r'C:/Users/USER/Downloads/Holmusk_analysis/bill_id.csv')
demographics = pd.read_csv(r'C:/Users/USER/Downloads/Holmusk_analysis/demographics.csv')
clinical_data = pd.read_csv(r'C:/Users/USER/Downloads/Holmusk_analysis/clinical_data.csv')

# Rename bill_id "patient_id" to avoid conflict
bill_id.rename(columns={"patient_id": "pt_id"}, inplace=True)

In [90]:
print("bill_id columns: " + str(bill_id.columns))
print("demographics columns: " + str(demographics.columns))
print("clinical_data columns: " + str(clinical_data.columns))
print(" ")
print("bill_id length: " + str(len(bill_id)))
print("demographics length: " + str(len(demographics)))
print("clinical_data length: " + str(len(clinical_data)))

bill_id columns: Index(['pt_id', 'bill_id', 'date_of_admission', 'amount'], dtype='object')
demographics columns: Index(['patient_id', 'gender', 'race', 'resident_status', 'date_of_birth'], dtype='object')
clinical_data columns: Index(['id', 'date_of_admission', 'date_of_discharge', 'medical_history_1',
       'medical_history_2', 'medical_history_3', 'medical_history_4',
       'medical_history_5', 'medical_history_6', 'medical_history_7',
       'preop_medication_1', 'preop_medication_2', 'preop_medication_3',
       'preop_medication_4', 'preop_medication_5', 'preop_medication_6',
       'symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'symptom_5',
       'lab_result_1', 'lab_result_2', 'lab_result_3', 'weight', 'height'],
      dtype='object')
 
bill_id length: 13600
demographics length: 3000
clinical_data length: 3400


bill_id actually has repeated patient ids, but with separate bill ids! Maybe we can just combine all the bill amounts together

Seems like bill_id has 4 rows for each patient visit. Patients that visit more than once however, will have 4 * n rows

In [91]:
# Find any repeated values
print("bill_id: " + str(len(bill_id["pt_id"].unique())) + " / " + str(len(bill_id)))
print("demographics: " + str(len(demographics["patient_id"].unique())) + \
      " / " + str(len(demographics)))
print("clinical_data: " + str(len(clinical_data["id"].unique())) + \
      " / " + str(len(clinical_data)))


bill_id: 3000 / 13600
demographics: 3000 / 3000
clinical_data: 3000 / 3400


Now let's combine the bill_ids into the correct format.

We add date_of_admission to their bill_ids to separate multiple visits by the same patient. Of course, we can look at the impact of multiple visits on the bill amount

In [121]:
bill_id_2 = bill_id.copy(deep=True)

# Save new id as id_w_date so we can reuse id to add in demographic info
bill_id_2["id_w_date"] = bill_id["pt_id"] + bill_id["date_of_admission"]

# bill_id_2["pt_id"][0]
len(bill_id_2["id_w_date"].unique())

3400

In [122]:
bill_id_2 = bill_id_2.sort_values("id_w_date")

# Maybe we can check the vist number too!
admission_count = []

# Combine the bill amounts
bill = []
current = bill_id_2["id_w_date"][0]
total_bill = 0
count = 1
for index, row in bill_id_2.iterrows():
    if row["id_w_date"] == current:
        total_bill += row["amount"]
        count += 1
    else:
        bill.append(total_bill)
        admission_count.append((count // 4) + 1)
        count = 0
        current = row["id_w_date"]
        total_bill = row["amount"]
bill.append(total_bill)
admission_count.append(count)

print(len(bill))
print(len(admission_count))

# Bill has been combined!

3400
3400


In [116]:
12 // 4

3

Let's do the same for clinical_data, and sort it, and add in the bill.

For clinical_data, we actually see that some patients have multiple logs, because they enter the hospital multiple times! We must treat those as separate values.

In [123]:
# clinical_data processing
cd_2 = clinical_data.copy(deep=True)
cd_2["id_w_date"] = clinical_data["id"] + clinical_data["date_of_admission"]

# Sort by new patient id
cd_2 = cd_2.sort_values("id_w_date")

# Add in total_bill
cd_2["total_bill"] = bill
cd_2["admission_count"] = admission_count

Now let's add in the relevant demographic data into cd_2

In [126]:
demographics.columns

Index(['patient_id', 'gender', 'race', 'resident_status', 'date_of_birth'], dtype='object')

In [147]:
# Add empty columns to cd_2
gender = []
race = []
citizenship = []
dob = []

# Check patient id in demographic, and add to cd_2
count_cd2 = 0
count_demo = 0
loop = 0
while count_cd2 < 3400 and count_demo < 3000:
#     print(loop)
#     loop += 1
    if cd_2["id"][count_cd2] == demographics["patient_id"][count_demo]:
        gender.append(demographics["gender"][count_demo])
        race.append(demographics["race"][count_demo])
        citizenship.append(demographics["resident_status"][count_demo])
        dob.append(demographics["date_of_birth"][count_demo])
        count_cd2 += 1
    else: 
        count_demo += 1
        
print(len(gender))

# Add the columns
cd_2["gender"] = gender
cd_2["race"] = race
cd_2["citizenship"] = citizenship
cd_2["date_of_birth"] = dob

3400


In [148]:
print(cd_2["gender"])

0       Female
1       Female
2         Male
3       Female
4         Male
         ...  
3395    Female
3396      Male
3397      Male
3398      Male
3399    Female
Name: gender, Length: 3400, dtype: object


Now we compile into our full dataset!

In [161]:
raw_df = cd_2.copy(deep=True)

In [150]:
print(raw_df.columns)

Index(['id', 'date_of_admission', 'date_of_discharge', 'medical_history_1',
       'medical_history_2', 'medical_history_3', 'medical_history_4',
       'medical_history_5', 'medical_history_6', 'medical_history_7',
       'preop_medication_1', 'preop_medication_2', 'preop_medication_3',
       'preop_medication_4', 'preop_medication_5', 'preop_medication_6',
       'symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'symptom_5',
       'lab_result_1', 'lab_result_2', 'lab_result_3', 'weight', 'height',
       'id_w_date', 'total_bill', 'admission_count', 'gender', 'race',
       'citizenship', 'date_of_birth'],
      dtype='object')


### Make the column values uniform and consistent

Now let's clean up the data and make it code-readable

In [151]:
# Check NA
for column in raw_df.columns:
    if raw_df[column].isnull().values.any():
        print(column + " nulls: " + str(raw_df[column].isnull().values.any()))
        
# Medical_history 2 and 5 have nulls. Let's check the number
print(raw_df["medical_history_2"].isnull().sum())
print(raw_df["medical_history_5"].isnull().sum())

medical_history_2 nulls: True
medical_history_5 nulls: True
233
304


There is a significant number of nulls. While I think the dataset is big enough where we can just remove them, I want to see if there is any reason behind these nulls. 

So let's filter them out, and analyse them separately (later...)

In [164]:
# Rectify gender - no nulls
# Male = 1, female = 0
gender_df = raw_df.copy(deep=True)
# print(gender_df["gender"].isnull().sum())

gender_df["gender"] = np.where(gender_df["gender"].str.lower().str.startswith("m"), 1, 0)

print(len(gender_df.loc[gender_df["gender"] == 0]))

1702


In [165]:
# Update gender
raw_df["gender"] = gender_df["gender"]

In [168]:
# Add in age column
age_df = raw_df.copy(deep=True)
# print(age_df["date_of_birth"].isnull().sum()) # No nulls for both
# print(age_df["date_of_admission"].isnull().sum())

doa = age_df["date_of_admission"].str.split('/', expand=True).astype(int)
dob = age_df["date_of_birth"].str.split('/', expand=True).astype(int)

age_at_admission = [a - b for a, b in zip(doa[2], dob[2])]

# Check that there are no negative ages
count = 0
for age in age_at_admission:
    if age < 0:
        count += 1
print("Negative ages: " + str(count))
print(len(age_at_admission))

Negative ages: 0
3400


In [169]:
raw_df["age_at_admission"] = age_at_admission

In [171]:
# Let's change date_of_admission and date_of_birth into month and year values
yoa, moa = doa[2], doa[1]
yob, mob = dob[2], dob[1]

raw_df["year_of_admission"] = yoa
raw_df["month_of_admission"] = moa
raw_df["year_of_birth"] = yob
raw_df["month_of_birth"] = mob

In [172]:
# Check all newly-formatted columns
raw_df["year_of_birth"]

0       1983
1       1943
2       1972
3       1976
4       1942
        ... 
3395    1976
3396    1963
3397    1938
3398    1950
3399    1956
Name: year_of_birth, Length: 3400, dtype: int32

In [180]:
# Change date_of_discharge to duration_of_stay
# We make use of datetime object to do this for us
# Date format is  DD/MM/YYYY
from datetime import date

duration_of_stay = []

for index, row in raw_df.iterrows():
    _doa = row["date_of_admission"].split('/')
    _dod = row["date_of_discharge"].split('/')
    d_a = date(int(_doa[2]), int(_doa[1]), int(_doa[0]))
    d_d = date(int(_dod[2]), int(_dod[1]), int(_dod[0]))
    duration_of_stay.append((d_d - d_a).days)
    
# Check negative duration
count = 0
for day in duration_of_stay:
    if day < 0:
        count += 1
        
print("Negative durations: " + str(count))
print(len(duration_of_stay))

Negative durations: 0
3400


Next, handle missing values for medical condition

### Lastly, let's clean up the data and remove unwanted columns

In [30]:
# Remove duplicate unneeded columns at the end
raw_df = raw_df.drop(columns=["pt_id", "id"])
print(raw_df.columns)

KeyError: "['pt_id' 'id'] not found in axis"

### Export data

Lets export this full dataset, and then shift the columns

In [34]:
# raw_df.to_csv("raw_df.csv")